In [3]:
from data_loader import data_loader_v1

x_train, x_val, y_train, y_val = data_loader_v1("./dataset/train/train.csv", output_size=1)

In [4]:
from catboost import CatBoostRegressor

jjjjjjjj

In [27]:
from xgboost import XGBRegressor

xgb_params = {
    'n_estimators': 1000,
    'random_state': 2024,
    "learning_rate": 0.01,
    'max_depth': 10
}

In [28]:
from sklearn.ensemble import VotingRegressor

In [5]:
pred = {}
models = {}
for item in x_train.keys():
    models[item] = CatBoostRegressor(**cat_params)
    models[item].fit(x_train[item], y_train[item])
    pred[item] = models[item].predict(x_val[item])

0:	learn: 129284.8397689	total: 56ms	remaining: 55.9s
999:	learn: 224.2581003	total: 3.98s	remaining: 0us
0:	learn: 14397.0522418	total: 7.49ms	remaining: 7.48s
999:	learn: 12.8912030	total: 5.3s	remaining: 0us
0:	learn: 6211.2534509	total: 6.81ms	remaining: 6.8s
999:	learn: 14.7293540	total: 5.17s	remaining: 0us
0:	learn: 26239.9381280	total: 5.76ms	remaining: 5.75s
999:	learn: 296.5634010	total: 4.09s	remaining: 0us
0:	learn: 5575.7622954	total: 6.76ms	remaining: 6.76s
999:	learn: 1.9527895	total: 5.63s	remaining: 0us
0:	learn: 397.8227900	total: 6.21ms	remaining: 6.21s
999:	learn: 0.1532758	total: 5.08s	remaining: 0us
0:	learn: 4326.9008554	total: 8.13ms	remaining: 8.12s
999:	learn: 2.0668536	total: 5.5s	remaining: 0us
0:	learn: 329.2509113	total: 6.76ms	remaining: 6.76s
999:	learn: 0.1906828	total: 5.52s	remaining: 0us
0:	learn: 961.4188243	total: 10.8ms	remaining: 10.8s
999:	learn: 0.6808010	total: 5.25s	remaining: 0us
0:	learn: 4671.6756164	total: 6.77ms	remaining: 6.76s
999:	lea

In [75]:
submission = pd.read_csv('./sample_submission.csv')
zeros = [1]*75
submission['감자'] = zeros

In [76]:
data_case = [
    ["건고추", "화건", "30 kg", "상품"],
    ["감자", "감자 수미", "20키로상자", "상"],
    ["배", "신고", "10 개", "상품"],
    ["깐마늘(국산)", "깐마늘(국산)", "20 kg", "상품"],
    ["무", "무", "20키로상자", "상"],
    ["상추", "청", "100 g", "상품"],
    ["배추", "배추", "10키로망대", "상"],
    ["양파", "양파", "1키로", "상"],
    ["대파", "대파(일반)", "1키로단", "상"]
]

pred = {'건고추': [], '감자': [], '배': [], '깐마늘(국산)': [], '무': [], '상추': [], '배추': [], '양파': [], '대파': []}

for i in range(25):
    time = f"TEST_{str(i).zfill(2)}"
    test = pd.read_csv(f'./dataset/test/{time}.csv')

    for case in range(9):
        x = test.loc[(test['품목명'] == data_case[case][0]) & 
                 (test['품종명'] == data_case[case][1]) &
                 (test['거래단위'] == data_case[case][2]) &
                 (test['등급'] == data_case[case][3])]
        x = x['평균가격(원)'].to_numpy()
        pred[data_case[case][0]].append(models[case].predict(x))

for case in range(9):
    pred[data_case[case][0]] = np.array(pred[data_case[case][0]]).flatten()

In [77]:
data_case = [
    ["사과", "홍로", "10 개", "상품"],
    ["사과", "후지", "10 개", "상품"]
]

pred['사과'] = []
for i in range(25):
    time = f"TEST_{str(i).zfill(2)}"
    test = pd.read_csv(f'./dataset/test/{time}.csv')
    
    x = test.loc[(test['품목명'] == '사과') & 
             ((test['품종명'] == '홍로') | (test['품종명'] == '후지')) &
             (test['거래단위'] == '10 개') &
             (test['등급'] == '상품')]

    x = x['평균가격(원)'].to_numpy()
    pred['사과'].append(models[9].predict(x))
        
pred['사과'] = np.array(pred['사과']).flatten()

In [78]:
for key in pred.keys():
    submission[key] = pred[key]

In [79]:
submission

,시점,감자,건고추,깐마늘(국산),대파,무,배추,사과,상추,양파,배
0,TEST_00+1순,27384.443731,721644.251881,37881.535138,1911.231245,85398.392488,1265.046447,3836.054177,8290.623370,3748.660936,40167.719484
1,TEST_00+2순,27650.126980,706412.568276,35875.972627,1863.571737,86799.540840,1039.900324,3679.636000,8346.885420,4759.107931,42769.478496
2,TEST_00+3순,28136.619614,696350.024464,35280.623467,1775.260664,93003.991957,908.436369,3509.590522,8339.541567,5539.914060,42940.029533
3,TEST_01+1순,27384.443731,621389.379107,37881.535138,1920.987364,85398.392488,1265.046447,3836.054177,8290.623370,3748.660936,26971.072549
4,TEST_01+2순,27650.126980,638090.892316,35875.972627,1952.844241,86799.540840,1039.900324,3679.636000,8346.885420,4759.107931,27871.195539
...,...,...,...,...,...,...,...,...,...,...,...
70,TEST_23+2순,27650.126980,637912.277516,35875.972627,1960.560592,86799.540840,1039.900324,3679.636000,8346.885420,4759.107931,31822.895381
71,TEST_23+3순,28136.619614,624551.531758,35280.623467,1731.340778,93003.991957,908.436369,3509.590522,8339.541567,5539.914060,31722.331906
72,TEST_24+1순,27384.443731,476690.101100,37881.535138,1551.872183,85398.392488,1265.046447,3836.054177,8290.623370,3748.660936,39137.786148
73,TEST_24+2순,27650.126980,473009.428343,35875.972627,1634.368397,86799.540840,1039.900324,3679.636000,8346.885420,4759.107931,42639.282959


In [80]:
submission.to_csv('submit.csv', index=False)